In [1]:
import dao.MongoUsersSong as mus
import panda.SongUser as supd 
# 重要信息，此程序需要再user_tag_recom_main.py 和 user_song_recom_main.py后运行
# 通过用户进行歌曲的歌曲相似度推荐 获取的其中的相似歌曲
mongomus = mus.UserSongRecomAfterSongSimilarity(
    user_id="1a78ac10b67911ed95dd00155dadb10b")
# 1a78ac10b67911ed95dd00155dadb10b
# mongomus.makeRecomBySongSimilarityAnswer(limit="ONE")
# 创建程序进程

In [2]:
# 程序从这里开始
# ONE 表示只操作目标用户，ALL 时 
# 表示目标用户的id 不再重要，会记录所有用户的歌曲之间的相似度
# 同时ALL时 是将数据存储到song 中，记录为SimilaritySongs
# ONE，是将数据存储到recom 中，记录为RecomSong
# mongomus.makeRecomBySongSimilarityAnswer(limit="ONE")
# 此处暂时使用ONE 标签的程序过程
# 此处sign 适用于判断是输出哪种格式
# mongomus.makeRecomAnswerSong(sign=1)
# 首先是ONE 和 ALL 两种模式的共同方法

# 获取recom中之前设置的数据
projections={"_id": 0, "id": 1, "name": 1, "RecomUsers": 1}
doc=mongomus.rdb.findDocument(
        collection_name = "recom", query = mongomus.query, projection = projections)
# 储存请求用户的名字
mongomus.usr.user_name=doc[0]['name']
# 跟据数据库给的RecUsers来处理数据
list_users_id=[]
list_users_id.append(mongomus.user_id)
# print(self.user_id, list_users_id)
# users_id 合集，之后会通过这个来形成x轴坐标
list_users_id.extend(doc[0]['RecomUsers'])
# 此处生成list_users_id
print(len(list_users_id))
for item in list_users_id:
    print(item)
    break


43
1a78ac10b67911ed95dd00155dadb10b


In [3]:
# 构成xy轴的内容源头
docs=mongomus.usr.getSongsWithUserIdFromMongo(array = list_users_id)

In [4]:
# xy稀疏矩阵信息,显示docs 的长度
# print(type(docs))
print(len(docs))


43


In [5]:
# matrix_data = data
# pandas_x = self.x
# pandas_y = self.y
data=mongomus.usr.makeXYMatric(docs = docs)
# data 存储的是每个歌曲信息的坐标位置信息
# for item in data:
#     print(data)
#     break
# 生成pandas
sudf=supd.SUPandas(datas = list(
    map(list, data.toarray())), items = mongomus.usr.y, users = mongomus.usr.x)

In [6]:
# 最后构成的稀疏矩阵
sudf.df

,RuRuUIH,s7UIjlHxS,WfTR2,H298YN,mdOQjX,m3NOW7Pp,G,Z,c4D,X2Bwh38,...,KvC,zVZzhaw,1C3ijya,BJ1HR0,yNHtXCz5M,MGrVl,FnJi,GcTaF8d,afqmIx79w,dj775w
491529,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
567083018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29097995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29196302,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
516358164,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1540081,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
400162805,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1540086,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1540090,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# 寻找用户的推荐歌曲
projections = {"_id": 0, "songs": 1}
# dict_song = mongomus.makeRecomSongDictionForUser(
#     df_object=sudf, user_songs=mongomus.rdb.findDocument(
#         collection_name="like", query=mongomus.query, projection=projections)[0]['songs'])
# 获得目标用户的songs
user_songs = mongomus.rdb.findDocument(collection_name="like", query=mongomus.query, projection=projections)
# 展示第一条数据
print(user_songs[0]['songs'][0])
user_songs = user_songs[0]['songs']

{'id': 1370027704, 'name': '复甦（中式医乐·黄钟宫五正调）', 'union': [{'tags': ['轻音乐', '浪漫', '经典']}]}


In [8]:
similar_item = sudf.makeSimilarityBetweenItem()
similar_user = sudf.makeSimilarityBetweenUser()

/home/arki/.local/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/home/arki/.local/lib/python3.10/site-packages/sklearn/metrics/pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [9]:
similar_item

,491529,567083018,29097995,29196302,516358164,28377108,516358165,1303019546,1456603168,1605671,...,1476132827,28868577,32309218,1377501157,2326503,1540081,400162805,1540086,1540090,65533
491529,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.333333,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0
567083018,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29097995,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29196302,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.333333,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
516358164,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1540081,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
400162805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1540086,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.5,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1540090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
similar_user

,RuRuUIH,s7UIjlHxS,WfTR2,H298YN,mdOQjX,m3NOW7Pp,G,Z,c4D,X2Bwh38,...,KvC,zVZzhaw,1C3ijya,BJ1HR0,yNHtXCz5M,MGrVl,FnJi,GcTaF8d,afqmIx79w,dj775w
RuRuUIH,1.000000,0.011715,0.010700,0.013115,0.016113,0.020774,0.010799,0.015355,0.007330,0.015496,...,0.011655,0.023783,0.011936,0.007614,0.007290,0.021958,0.016895,0.013873,0.007716,0.015831
s7UIjlHxS,0.011715,1.000000,0.013915,0.005988,0.013346,0.017937,0.015385,0.008160,0.012961,0.009737,...,0.012075,0.011616,0.008642,0.009524,0.010297,0.023601,0.010934,0.013058,0.005690,0.009804
WfTR2,0.010700,0.013915,1.000000,0.003958,0.014085,0.027708,0.016008,0.011794,0.019373,0.014414,...,0.011000,0.019474,0.012291,0.011879,0.022082,0.014493,0.015611,0.014925,0.006321,0.012195
H298YN,0.013115,0.005988,0.003958,1.000000,0.005502,0.007528,0.007657,0.005155,0.007364,0.008584,...,0.005111,0.008052,0.004141,0.003891,0.001818,0.005995,0.001462,0.006723,0.005362,0.004082
mdOQjX,0.016113,0.013346,0.014085,0.005502,1.000000,0.011864,0.016473,0.013853,0.012241,0.015755,...,0.019751,0.013958,0.008055,0.008899,0.019481,0.015690,0.011278,0.017472,0.007895,0.020809
m3NOW7Pp,0.020774,0.017937,0.027708,0.007528,0.011864,1.000000,0.013562,0.015523,0.015044,0.012132,...,0.008621,0.014911,0.008511,0.009278,0.007952,0.017188,0.013239,0.013359,0.007212,0.018124
G,0.010799,0.015385,0.016008,0.007657,0.016473,0.013562,1.000000,0.016667,0.014170,0.018924,...,0.019080,0.013963,0.015171,0.010909,0.009292,0.015803,0.016211,0.009912,0.013158,0.008717
Z,0.015355,0.008160,0.011794,0.005155,0.013853,0.015523,0.016667,1.000000,0.014440,0.014172,...,0.011788,0.009470,0.010929,0.011640,0.013292,0.019124,0.010782,0.007759,0.007417,0.014146
c4D,0.007330,0.012961,0.019373,0.007364,0.012241,0.015044,0.014170,0.014440,1.000000,0.013527,...,0.010823,0.012526,0.008516,0.005848,0.014739,0.019828,0.011799,0.007479,0.009831,0.012107
X2Bwh38,0.015496,0.009737,0.014414,0.008584,0.015755,0.012132,0.018924,0.014172,0.013527,1.000000,...,0.010582,0.019547,0.011905,0.011481,0.014396,0.016034,0.012536,0.014737,0.006803,0.009423


In [12]:
# 生成用于存储的结果
# mongomus.makeRecomSongDictionForUser(
#     df_object=sudf, user_songs=user_songs)

diction = sudf.makeTopNSongs(
    # 对用户收藏的歌曲，每首获取最相似的前5首
    #  将这个数据进行几个合并，
    # 然后将数据进行随机提取，
    # 提取其中的 前5个
    similar=similar_item,
    song_cnt=5)
# 此处显示获取到每首歌的相似度前5首歌曲
print(diction)

{491529: [1449559810, 1874946820, 1427912609, 419374614, 27834841], 567083018: [729836, 4017601, 532333, 1509208, 424057262], 29097995: [22822318, 503426999, 21973183, 468932689, 534065501], 29196302: [417833239, 17670016, 3781857, 29586022, 465341538], 516358164: [590672, 1476449694, 2531854, 544914961, 516823327], 28377108: [526815221, 29825389, 1464238286, 1403764384, 28912536], 516358165: [5165319, 29497924, 29133639, 513617762, 28547331], 1303019546: [94730, 32408771, 5200330, 473208560, 497458504], 1456603168: [1667907, 25731276, 19805834, 410714379, 19574402], 1605671: [4064268, 1417864875, 472848587, 27515168, 17690713], 26116136: [1349964387, 27737240, 36947823, 5253734, 28556338], 622645: [28853535, 19375084, 27640064, 5052232, 540985273], 327736: [1408713429, 405167769, 434659433, 5195497, 529824781], 23986232: [534228810, 3608060, 519913064, 1816789527, 26493567], 31260730: [1454133413, 776668, 1405376096, 1870580212, 29747622], 16646207: [1907443034, 447120862, 30070395, 1

In [14]:
# 这个方法获取的是用户自自身收藏的歌曲的歌曲
list_songs = mongomus.makeSongIdList(user_songs)
print(len(list_songs))
print(list_songs)

448
[1370027704, 331160, 1324244054, 19153845, 510515495, 17358564, 183679, 29119025, 24615861, 507585627, 31192913, 5027852, 554024777, 525278875, 472716484, 19078789, 36226193, 488026985, 454698089, 553901186, 407677975, 28748595, 744932, 173466, 175246, 28977397, 29133639, 1306273863, 458356832, 28959185, 447578836, 1308782735, 499165825, 1356668748, 464797841, 404166555, 5057816, 571585964, 481014963, 22774088, 1461521, 528864917, 1311535641, 22498860, 5058719, 463031, 410181618, 26123237, 679645, 27074449, 22064605, 29418350, 20781240, 591071, 35504129, 27591589, 28785381, 2003591, 1325896815, 29100034, 1801563, 31370984, 16437022, 1476449694, 636692, 27506128, 4425889, 29418323, 495857412, 1059071, 21697173, 476515542, 204217, 442898049, 479813913, 256468, 18663058, 4007275, 28889785, 1693715, 2531854, 26584453, 420513233, 28838470, 26123464, 34045202, 5168414, 17594193, 19277298, 468998937, 29712208, 1883224, 27443402, 553715430, 413834666, 29284485, 529500547, 28240590, 1887366

In [15]:
# 此处运算通过删除用户已经收藏的歌曲，然后再随机获取得到结果
list_songs_result = mongomus.makeSongsFromSimilarDict(
    diction=diction, songs=list_songs)
print(len(list_songs_result))
print(list_songs_result)

30
[31477891, 31426805, 25699062, 1474374809, 33856424, 458696351, 4038990, 1865046472, 446512251, 536624722, 1489749135, 1304015, 554407504, 28126710, 20998010, 1412661626, 27711117, 2193481, 2591727, 212430, 435872951, 5044714, 480409513, 5282045, 1367983733, 1350521355, 1233723, 1831252, 17579257, 1423608]


In [ ]:
# 假设我们 LIMIT == "ALL"
#　一下是程序中启动语句
mongomus_all = mus.UserSongRecomAfterSongSimilarity()

# mongomus_all.saveUserSongRecomAnswer(
#                 mongomus_all.makeRecomAnswerSong(
#                     sign=0), update_name="SimilaritySongs", collection="song")


In [ ]:
# 获取recom中之前设置的数据
projections = {"_id": 0, "id": 1, "name": 1, "RecomUsers": 1}
doc = mongomus_all.rdb.findDocument(
    collection_name="recom", query=mongomus_all.query, projection=projections)
# 储存请求用户的名字
mongomus_all.usr.user_name = doc[0]['name']
# 跟据数据库给的RecUsers来处理数据
list_users_id = []
list_users_id.append(mongomus_all.user_id)
# print(self.user_id, list_users_id)
# users_id 合集，之后会通过这个来形成x轴坐标
list_users_id.extend(doc[0]['RecomUsers'])
# 构成xy轴的内容源头
docs = mongomus_all.usr.getSongsWithUserIdFromMongo(array=list_users_id)
# matrix_data = data
# pandas_x = self.x
# pandas_y = self.y
data = mongomus_all.usr.makeXYMatric(docs=docs)
sudf = supd.SUPandas(datas=list(
    map(list, data.toarray())), items=mongomus_all.usr.y, users=mongomus_all.usr.x)

# 以上生成pandas 部分的代码基本一致
# 后面处理数据的方式不同，这里需要的是每首歌的相似歌曲，所以需要用户的信息


In [ ]:
# 寻找相似歌曲
dict_song = mongomus_all.makeSimilaritySongDictionForSong(
    df_object=sudf)
# 该方法中内容
# diction = df_object.makeTopNSongs(
#             # 对用户收藏的歌曲，每首获取最相似的前5首
#             # 将这个数据进行几个合并，
#             # 然后将数据进行随机提取，
#             # 提取其中的 前5个
#             similar=df_object.makeSimilarityBetweenItem(),
#             song_cnt=5)

In [ ]:
print(dict_song)